# Raining in MaNILa - Metaheuristic Algorithm

#### Data Preprocessing

In [ ]:
## Import necessary libraries
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_2018 = pd.read_csv(r'/content/drive/Shareddrives/DATASET raining in maNILa/kapagodpo.csv',
                 skipinitialspace=True)
df_2018.head()

,FlightDate,Month,TimeOfDay,Airline,Distance,ESTDepTotalMinutes,Route,AvgDelayPerRoute,DepDelayAvgPerMonth,Holiday,DepDelay
0,2020-01-01,1,Midnight,Delta Air Lines Inc.,1535.0,240.0,LAX-MSP,6.448685,8.521821,True,0.0
1,2020-01-01,1,Midnight,Delta Air Lines Inc.,196.0,300.0,FSD-MSP,6.249456,8.521821,True,0.0
2,2020-01-01,1,Midnight,Delta Air Lines Inc.,1533.0,300.0,FAI-SEA,2.816303,8.521821,True,0.0
3,2020-01-01,1,Midnight,Delta Air Lines Inc.,1590.0,309.0,SLC-ATL,8.642204,8.521821,True,3.0
4,2020-01-01,1,Midnight,Southwest Airlines Co.,277.0,315.0,HRL-HOU,6.403361,8.521821,True,1.0


In [ ]:
df_2018.shape

(3200090, 11)

In [ ]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200090 entries, 0 to 3200089
Data columns (total 11 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FlightDate           object 
 1   Month                int64  
 2   TimeOfDay            object 
 3   Airline              object 
 4   Distance             float64
 5   ESTDepTotalMinutes   float64
 6   Route                object 
 7   AvgDelayPerRoute     float64
 8   DepDelayAvgPerMonth  float64
 9   Holiday              bool   
 10  DepDelay             float64
dtypes: bool(1), float64(5), int64(1), object(4)
memory usage: 247.2+ MB


In [ ]:
## Convert distance from miles to kilometer
def to_km(val):
    return val*1.609344

#### Try pseudo routes

In [ ]:
df_2018['Route'].value_counts()

Route
MCO-ATL    10863
ATL-MCO    10647
TPA-ATL     9352
ATL-TPA     9270
FLL-ATL     8870
           ...  
LGA-CHS        1
JFK-RIC        1
DTW-GSO        1
IAD-LAS        1
CHS-LGA        1
Name: count, Length: 2676, dtype: int64

In [ ]:
routes = df_2018['Route'].values
routes

array(['LAX-MSP', 'FSD-MSP', 'FAI-SEA', ..., 'SFO-ATL', 'SEA-ATL',
       'LAX-JFK'], dtype=object)

In [ ]:
def separate_strings_with_dash(string_list):
    # Split each string in the list by "-" and flatten the result
    separated_strings = [substring for string in string_list for substring in string.split("-")]
    return separated_strings

In [ ]:
routes = separate_strings_with_dash(routes)
routes

['LAX',
 'MSP',
 'FSD',
 'MSP',
 'FAI',
 'SEA',
 'SLC',
 'ATL',
 'HRL',
 'HOU',
 'MSP',
 'ATL',
 'DEN',
 'PHX',
 'VPS',
 'ATL',
 'AUS',
 'STL',
 'HOU',
 'FLL',
 'BHM',
 'HOU',
 'TUL',
 'HOU',
 'MSY',
 'ATL',
 'MCI',
 'MCO',
 'MKE',
 'DEN',
 'BHM',
 'ATL',
 'PIT',
 'MCO',
 'OAK',
 'PHX',
 'BDL',
 'DTW',
 'MKE',
 'BNA',
 'CMH',
 'MCO',
 'ABQ',
 'HOU',
 'SAT',
 'STL',
 'DEN',
 'BNA',
 'MSP',
 'MDW',
 'HOU',
 'STL',
 'STL',
 'HOU',
 'CHS',
 'MDW',
 'MCI',
 'ATL',
 'ATW',
 'ATL',
 'OMA',
 'ATL',
 'LAX',
 'AUS',
 'SAT',
 'DEN',
 'PHX',
 'MDW',
 'BNA',
 'MCO',
 'AUS',
 'HOU',
 'SEA',
 'PHX',
 'CMH',
 'ATL',
 'STL',
 'ATL',
 'MSY',
 'ATL',
 'HOU',
 'ATL',
 'PHL',
 'BNA',
 'CLE',
 'ATL',
 'MCI',
 'DAL',
 'PNS',
 'ATL',
 'BOS',
 'DTW',
 'DTW',
 'MDW',
 'PDX',
 'MDW',
 'AUS',
 'MDW',
 'RNO',
 'LAX',
 'HOU',
 'TPA',
 'DTW',
 'ATL',
 'CLE',
 'BNA',
 'BOS',
 'ATL',
 'SLC',
 'DTW',
 'STL',
 'BWI',
 'ICT',
 'ATL',
 'STL',
 'FLL',
 'CID',
 'ATL',
 'AUS',
 'LAS',
 'EWR',
 'DTW',
 'HOU',
 'MDW',
 'LIT',


#### Device an algo to find all posible routes to take

In [ ]:
all = list(set(routes)) # removes repeated values
len(all), len(routes)

(164, 6400180)

In [ ]:
## Install additional libraries
!pip install haversine
!pip install airportsdata
!pip install mealpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.0/911.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 73.8 MB/s eta 0:00:00


In [ ]:
import airportsdata

def get_coordinates(geo_loc):
    airports = airportsdata.load('IATA')

    if geo_loc not in airports:
        return 0, 0
    return airports[geo_loc]['lat'], airports[geo_loc]['lon']

In [ ]:
coor = [get_coordinates(x) for x in all]
waypoint = dict(zip(all, coor))

In [ ]:
place = []
for key, val in waypoint.items():
    if (0,0) == val:
        place.append(key)
place

[]

In [ ]:
#waypoint['ISN'] = (48.1781, 103.6422) # Replaced the coordiantes with the real coordinates

<br><br><br>

#### Check the algo

In [ ]:
import numpy as np
from mealpy import PermutationVar, Problem
from haversine import haversine, Unit
import math
import random

class RouteFinder(Problem):
    def __init__(self, bounds=None, minmax="min", data=None):
        self.data = data
        self.eps = 1e10  # Penalty function for vertex with 0 connection
        super().__init__(bounds, minmax)

    # Calculate the fitness of an individual
    def obj_func(self, x):
        x_decoded = self.decode_solution(x)
        individual = x_decoded["path"]

        total_distance = 0
        for idx in range(len(individual) - 1):
            start_node = individual[idx]
            end_node = individual[idx + 1]
            weight = self.data[start_node, end_node]
            if weight == 0:
                return self.eps
            total_distance += weight
        return total_distance

class Meta(RouteFinder):
    def __init__(self, waypoint, orig, dest, model_params):
        self.waypoint = waypoint
        self.graph = {}
        self.orig = orig
        self.dest = dest
        self.model = model_params['model']
        self.epoch = model_params['epoch']
        self.pop_size = model_params['pop_size']
        #self.dropout = (random.randrange(20, 80))/100
        self.dropout = 0.60 # test case

        self.path = [0, 0]
        self.orig_to_dest = [0, 0]
        self.graph_mat = []
        self.num_vertices = len(self.graph_mat)
        self.check_dist = 0

        '''
        model_params = {model, epoch, population size}

        model - has to be a mealpy class (eg. ACOR, WOA, GA...)
        epoch - must be int type only
        population size - must also be int type
        '''

    ## Creates model
    def __create_model(self, graph):
        num_nodes = len(graph)
        bounds = PermutationVar(valid_set=list(range(0, num_nodes)), name="path")
        problem = RouteFinder(bounds=bounds, minmax="min", data=graph)
        model = self.model(self.epoch, self.pop_size)
        model.solve(problem)

        return model

    ## Validates traversal order (medyo weird naman kung nauna dest sa orgin)
    def __validate_order(self, model):
        check_model_dist = model.problem.decode_solution(model.g_best.solution)['path']
        return check_model_dist.index(self.orig_to_dest[0]) > check_model_dist.index(self.orig_to_dest[1])

    def __update_graph(self, new_graph):
        self.graph = new_graph

    def __update_graph_mat(self, new_mat):
        self.graph_mat = new_mat

    ## Generates graph from coordinates
    def __generate_graph(self, waypoint):
        graph = {}

        for city1, coords1 in waypoint.items():
            graph[city1] = {}
            for city2, coords2 in waypoint.items():
                if city1 != city2:
                    distance = haversine(coords1, coords2, unit=Unit.KILOMETERS)
                    graph[city1][city2] = round(distance, 2)

        self.graph = graph

    def __remove_edge(self, node1, node2):
        if node1 in self.graph and node2 in self.graph[node1]:
            del self.graph[node1][node2]
        if node2 in self.graph and node1 in self.graph[node2]:
            del self.graph[node2][node1]

    ## Creates a matrix from graph
    def __create_graph_matrix(self, graph):
        nodes = sorted(graph.keys())
        #node_to_idx = {node: i for i, node in enumerate(nodes)} # Find idx of each key, run in case na malito

        nodes = sorted(list(graph.keys()))
        num_nodes = len(nodes)
        graph_matrix = np.zeros((num_nodes, num_nodes))

        for i, node1 in enumerate(nodes):
            for j, node2 in enumerate(nodes):
                if node2 in graph[node1]:
                    graph_matrix[i][j] = graph[node1][node2]

        return graph_matrix

    ## Maps the index to the graph, traversing through the node
    def __map_result_to_graph(self, result):
        mapped_result = []
        for node_index in result:
            node_name = list(sorted(self.graph.keys()))[node_index]
            mapped_result.append(node_name)
        return mapped_result # returns list of char

    ## Calculates the distance from start node to end node
    def __calculate_distance(self, graph, nodes):
        total_distance = 0
        for i in range(len(nodes) - 1):
            current_node = nodes[i]
            next_node = nodes[i + 1]
            if current_node in graph and next_node in graph[current_node]:
                total_distance += graph[current_node][next_node]
            else:
                print(f"No direct connection between {current_node} and {next_node}.")
                total_distance += np.inf  # In case no edge, add infinity instead because... Why not?
        return total_distance # returns float

    ## Creates a new graph from the previous graph
    def __create_subgraph(self, graph, nodes_to_go_through):
        subgraph = {}
        for node in nodes_to_go_through:
            if node in graph:
                subgraph[node] = {}
                for neighbor, distance in graph[node].items():
                    if neighbor in nodes_to_go_through:
                        subgraph[node][neighbor] = distance

        # Remove edges between the first and last nodes
        if nodes_to_go_through[0] in subgraph and nodes_to_go_through[-1] in subgraph[nodes_to_go_through[0]]:
            del subgraph[nodes_to_go_through[0]][nodes_to_go_through[-1]]
        if nodes_to_go_through[-1] in subgraph and nodes_to_go_through[0] in subgraph[nodes_to_go_through[-1]]:
            del subgraph[nodes_to_go_through[-1]][nodes_to_go_through[0]]

        return subgraph

    ## Updates nodes
    def __update_start_end(self, new_start, new_end):
        self.orig_to_dest[0] = (new_start)
        self.orig_to_dest[1] = (new_end)

    ## Update params of the model
    def __update_params(self):
        if self.epoch > 5 and self.pop_size > 5:
            self.epoch = self.epoch - math.ceil(self.epoch*self.dropout)
            #self.pop_size = self.pop_size - math.ceil(self.pop_size*self.dropout)
        else:
          pass

    ## Update node idx
    def __update_node_idx(self):
        nodes = sorted(self.graph.keys())
        node_idx = {node: i for i, node in enumerate(nodes)}

        self.path[0] = node_idx[self.orig]
        self.path[1] = node_idx[self.dest]

    ## Distance checking with dijkstra
    def __dijkstra(self, graph, start, end):
        num_vertices = len(graph)
        shortest_distances = [float('inf')] * num_vertices
        shortest_distances[start] = 0
        visited = [False] * num_vertices
        path = [-1] * num_vertices

        for _ in range(num_vertices):
            min_distance = float('inf')
            min_index = -1
            for v in range(num_vertices):
                if not visited[v] and shortest_distances[v] < min_distance:
                    min_distance = shortest_distances[v]
                    min_index = v
            visited[min_index] = True
            for v in range(num_vertices):
                if not visited[v] and graph[min_index][v] != 0 and shortest_distances[min_index] + graph[min_index][v] < shortest_distances[v]:
                    shortest_distances[v] = shortest_distances[min_index] + graph[min_index][v]
                    path[v] = min_index

        shortest_path = []
        current = end
        while current != -1:
            shortest_path.append(current)
            current = path[current]
        shortest_path.reverse()

        return shortest_distances[end]


    ## Runs the code
    def run_meta(self):

        ## Initialize starting parameters
        self.__generate_graph(self.waypoint) ## Generate graph
        self.__remove_edge(self.orig, self.dest)

        mat = self.__create_graph_matrix(self.graph)
        self.__update_graph_mat(mat) # Updates graph into matrix
        self.__update_node_idx()
        self.__update_start_end(self.path[0], self.path[1])
        self.check_dist = self.__dijkstra(self.graph_mat, self.path[0], self.path[1])

        for _ in range(120):
            while True:
                model = self.__create_model(self.graph_mat)
                if not self.__validate_order(model):
                    break

            path = model.problem.decode_solution(model.g_best.solution)['path']
            path = path[path.index(self.orig_to_dest[0]):path.index(self.orig_to_dest[1])+1] # Takes the orig to dest route


            map_result = self.__map_result_to_graph(path)
            distance = self.__calculate_distance(self.graph, map_result) # Gets distance because... Why not?

            sub_graph = self.__create_subgraph(self.graph, map_result) # New graph created
            self.__update_graph(sub_graph)

            self.__update_node_idx()

            self.__update_start_end(self.path[0], self.path[1]) # update origin and dest
            self.__update_graph_mat(self.__create_graph_matrix(self.graph))
            self.__update_params()

            if distance <= self.check_dist:
              break

        return map_result, distance, self.check_dist

In [ ]:
orig, dest = 'ORD', 'DEN'

<br><br><br>

#### Final Testing

In [ ]:
from mealpy import ACOR, GA, PSO, BeesA, SA
import time

In [ ]:
## test for n times
run = 10

ant = ACOR.OriginalACOR
genetic = GA.BaseGA
particle = PSO.OriginalPSO
#sim = SA.OriginalSA
#bee = BeesA.OriginalBeesA

model_list = [ant, genetic, particle]
final_test = [[[] for _ in range(len(model_list))] for _ in range(run)]

model_name = ['ant', 'genetic', 'particle']

for i in range(run):
    for j in range(len(model_list)):
        params = {
          'model': model_list[j],
          'epoch': 100,
          'pop_size': 20
          }

        router = Meta(waypoint, orig, dest, params)
        start_time = time.time()

        res = router.run_meta()

        end_time = time.time()

        elapsed_time = end_time - start_time

        final_test[i][j] = (model_name[j], res, elapsed_time)

Streaming output truncated to the last 5000 lines.
INFO:mealpy.swarm_based.ACOR.OriginalACOR:>>>Problem: P, Epoch: 2, Current best: 3411.16, Global best: 3411.16, Runtime: 0.01016 seconds
INFO:mealpy.swarm_based.ACOR.OriginalACOR:Solving single objective optimization problem.
INFO:mealpy.swarm_based.ACOR.OriginalACOR:>>>Problem: P, Epoch: 1, Current best: 3411.16, Global best: 3411.16, Runtime: 0.01937 seconds
INFO:mealpy.swarm_based.ACOR.OriginalACOR:>>>Problem: P, Epoch: 2, Current best: 3411.16, Global best: 3411.16, Runtime: 0.01906 seconds
INFO:mealpy.swarm_based.ACOR.OriginalACOR:Solving single objective optimization problem.
INFO:mealpy.swarm_based.ACOR.OriginalACOR:>>>Problem: P, Epoch: 1, Current best: 3411.16, Global best: 3411.16, Runtime: 0.01754 seconds
INFO:mealpy.swarm_based.ACOR.OriginalACOR:>>>Problem: P, Epoch: 2, Current best: 3411.16, Global best: 3411.16, Runtime: 0.01918 seconds
INFO:mealpy.swarm_based.ACOR.OriginalACOR:Solving single objective optimization proble

In [ ]:
final_test

[[('ant', (['ORD', 'BZN', 'DEN'], 2746.0, 1425.75), 58.3871328830719),
  ('genetic', (['ORD', 'DAL', 'DEN'], 2332.34, 1425.75), 11.025912761688232),
  ('particle', (['ORD', 'ABQ', 'DEN'], 2358.38, 1425.75), 13.796910047531128)],
 [('ant',
   (['ORD', 'STL', 'DEN'], 1650.6399999999999, 1425.75),
   127.1900691986084),
  ('genetic', (['ORD', 'COS', 'DEN'], 1580.02, 1425.75), 12.737093210220337),
  ('particle', (['ORD', 'OKC', 'DEN'], 1910.22, 1425.75), 15.023838520050049)],
 [('ant',
   (['ORD', 'GRB', 'MSP', 'DEN'], 1776.7199999999998, 1425.75),
   297.0821924209595),
  ('genetic',
   (['ORD', 'FAR', 'DEN'], 1903.1599999999999, 1425.75),
   19.230761528015137),
  ('particle', (['ORD', 'OMA', 'DEN'], 1425.75, 1425.75), 11.26962661743164)],
 [('ant',
   (['ORD', 'PVD', 'BTV', 'DEN'], 4331.030000000001, 1425.75),
   207.77279567718506),
  ('genetic', (['ORD', 'CID', 'DEN'], 1426.54, 1425.75), 16.158443450927734),
  ('particle', (['ORD', 'ROC', 'DEN'], 3115.41, 1425.75), 12.392560005187988)

#### Find the algo with lowest distance

In [ ]:
temp, algo = [[0 for _ in range(len(model_list))] for _ in range(run)], [['' for _ in range(len(model_list))] for _ in range(run)]
for i in range(run):
    for j in range(len(model_list)):
        temp[i][j] = final_test[i][j][1][1]
        algo[i][j] = final_test[i][j][0]

combined_sets = []
for algo_list, temp_list in zip(algo, temp):
    combined_set = {(x, y) for x, y in zip(algo_list, temp_list)}
    combined_sets.append(combined_set)

combined_sets

[{('ant', 2746.0), ('genetic', 2332.34), ('particle', 2358.38)},
 {('ant', 1650.6399999999999), ('genetic', 1580.02), ('particle', 1910.22)},
 {('ant', 1776.7199999999998),
  ('genetic', 1903.1599999999999),
  ('particle', 1425.75)},
 {('ant', 4331.030000000001), ('genetic', 1426.54), ('particle', 3115.41)},
 {('ant', 1735.52), ('genetic', 1650.6399999999999), ('particle', 2516.55)},
 {('ant', 2594.49), ('genetic', 2141.79), ('particle', 2141.33)},
 {('ant', 4259.1), ('genetic', 1733.23), ('particle', 3052.17)},
 {('ant', 1903.1599999999999), ('genetic', 1500.4), ('particle', 2457.73)},
 {('ant', 3411.16), ('genetic', 1991.0), ('particle', 1545.8)},
 {('ant', 1519.24), ('genetic', 3644.35), ('particle', 2141.33)}]

In [ ]:
min_values = []
min_strings = []
for combined_set in combined_sets:
    min_value = float('inf')
    min_string = ''
    for string, value in combined_set:
        if value != float('inf') and value < min_value:
            min_value = value
            min_string = string
    min_values.append(min_value)
    min_strings.append(min_string)

print("Best Score:", min_values)
print("By Algo:", min_strings)

Best Score: [2332.34, 1580.02, 1425.75, 1426.54, 1650.6399999999999, 2141.33, 1733.23, 1500.4, 1545.8, 1519.24]
By Algo: ['genetic', 'genetic', 'particle', 'genetic', 'genetic', 'particle', 'genetic', 'genetic', 'particle', 'ant']


In [ ]:
resulting_factor = [{x, y} for x, y in zip(min_strings, min_values)]
resulting_factor

[{2332.34, 'genetic'},
 {1580.02, 'genetic'},
 {1425.75, 'particle'},
 {1426.54, 'genetic'},
 {1650.6399999999999, 'genetic'},
 {2141.33, 'particle'},
 {1733.23, 'genetic'},
 {1500.4, 'genetic'},
 {1545.8, 'particle'},
 {1519.24, 'ant'}]

#### Lowest distance

In [ ]:
from collections import Counter

# Count occurrences of each element in the list
counts = Counter(min_strings)

# Find the most common value and its count
most_common_value = counts.most_common(1)[0][0]
count_of_most_common_value = counts[most_common_value]

print("Best algo:", most_common_value)
print("Occurences:", count_of_most_common_value)

Best algo: genetic
Occurences: 6


In [ ]:
from collections import Counter

counts = Counter(min_strings)

sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
ranked_values = {value: rank + 1 for rank, (value, count) in enumerate(sorted_counts) if count > 1}

print("Repeated values ranked by frequency:")
for value, rank in ranked_values.items():
    print(f"Value: {value}, Rank: {rank}")

Repeated values ranked by frequency:
Value: genetic, Rank: 1
Value: particle, Rank: 2


#### Results

In [ ]:
final_test[0][1][1][:]

(['ORD', 'DAL', 'DEN'], 2332.34, 1425.75)

**Metaheuristic Algorithms: Ant, Genetic, Particle**

In [ ]:
## TIME & DISTANCE
idx = 0

while idx <= 2:
    average_time, average_dist = [], []
    for i in range(run):
        average_dist.append(final_test[i][idx][1][1])

    for i in range(run):
        average_time.append(final_test[i][idx][2])

    print(f'{model_name[idx]} | Average Exec. time: {(sum(average_time)/run):.2f}, Average Distance: {(sum(average_dist)/run):.2f}')
    idx+=1
    if idx > 2:
        break

ant | Average Exec. time: 150.78, Average Distance: 2592.71
genetic | Average Exec. time: 14.73, Average Distance: 1990.35
particle | Average Exec. time: 13.63, Average Distance: 2266.47


| Algorithm | Execution Time (s) | Distance (km)|
| :-- | :-: | :-: |
| Ant Colony Optimization | 150.78 | 2592.71 |
| Genetic Algorithm | 14.73 | 1990.35 |
| Particle Swarm Optimization | 13.63 | 2266.47 |

The table above shows the average values made by the algorithm, they were chosen based on the minimum values they gave; this test was repeated 10 times (Error fixed: The first table was a mistake because index was not moving)